In [55]:
import xml.etree.ElementTree as ET
from shapely.geometry import LineString
import numpy as np

In [57]:
def generate_maze(maze_layout, start_positions=None, goals=None):
    
    num_rows = len(maze_layout)
    num_cols = len(maze_layout[0]) if num_rows > 0 else 0

    added_walls = []

    def add_wall(x1, y1, x2, y2):
        wall = tuple([(x1, y1), (x2, y2)])
        if wall not in added_walls:
            added_walls.append(wall)

    # Generate internal walls based on the maze layout
    for i, row in enumerate(maze_layout):
        for j, cell in enumerate(row):
            cell_x = j - num_cols / 2 + 0.5  # Center x position of the cell
            cell_y = num_rows / 2 - i - 0.5  # Center y position of the cell

            if cell[0] == 1:  # North wall
                if i == 0:  # Top row cells
                    add_wall(cell_x-0.5, cell_y+0.5, cell_x+0.5, cell_y+0.5)
                elif maze_layout[i-1][j][2] == 0:  # If no South wall in the cell above, add North wall
                    add_wall(cell_x-0.5, cell_y+0.5, cell_x+0.5, cell_y+0.5)

            if cell[1] == 1:  # East wall
                add_wall(cell_x+0.5, cell_y+0.5, cell_x+0.5, cell_y-0.5)

            if cell[2] == 1:  # South wall
                add_wall(cell_x-0.5, cell_y-0.5, cell_x+0.5, cell_y-0.5)

            if cell[3] == 1:  # West wall
                if j == 0:  # Leftmost column cells
                    add_wall(cell_x-0.5, cell_y+0.5, cell_x-0.5, cell_y-0.5)
                elif maze_layout[i][j-1][1] == 0:  # If no East wall in the cell to the left, add West wall
                    add_wall(cell_x-0.5, cell_y+0.5, cell_x-0.5, cell_y-0.5)

    return added_walls


In [63]:
def are_parallel(line1, line2):
    """Check if two lines are parallel by comparing their direction vectors."""
    x1, y1 = line1.coords[0]
    x2, y2 = line1.coords[-1]
    x3, y3 = line2.coords[0]
    x4, y4 = line2.coords[-1]

    # Calculate direction vectors
    vector1 = (x2 - x1, y2 - y1)
    vector2 = (x4 - x3, y4 - y3)

    # Calculate cross product (should be zero if vectors are parallel)
    cross_product = vector1[0] * vector2[1] - vector1[1] * vector2[0]

    return cross_product == 0

def are_touching(line1, line2):
    """Check if two lines are touching (end of one is start of another)."""
    return line1.coords[-1] == line2.coords[0]

def combine_walls_if_parallel_and_touching(walls):
    combined_walls = []
    while walls:
        current_wall = walls.pop(0)
        i = 0
        while i < len(walls):
            if are_parallel(current_wall, walls[i]) and are_touching(current_wall, walls[i]):
                current_wall = LineString(list(current_wall.coords) + list(walls[i].coords[1:]))
                walls.pop(i)
            else:
                i += 1
        combined_walls.append(current_wall)
    return combined_walls

def walls_and_positions_to_xml(combined_walls, start_positions=None, goals=None):
    root = ET.Element("world")
    for wall in combined_walls:
        (x1, y1), (x2, y2) = wall.coords[0], wall.coords[-1]
        ET.SubElement(root, "wall", x1=str(x1), y1=str(y1), x2=str(x2), y2=str(y2))

    if start_positions:
        start_positions_element = ET.SubElement(root, "startPositions")
        for pos in start_positions:
            ET.SubElement(start_positions_element, "pos", x=str(pos["x"]), y=str(pos["y"]), theta=str(pos["theta"]))

    if goals:
        for goal in goals:
            ET.SubElement(root, "goal", id=str(goal["id"]), x=str(goal["x"]), y=str(goal["y"]))

    return ET.tostring(root, encoding='unicode', method='xml')

In [83]:
import xml.dom.minidom
def save_maze_to_file(xml_string, file_name):
    if not file_name.endswith('.xml'):
        file_name += '.xml'
    
    # Parse the XML string into a DOM object
    dom = xml.dom.minidom.parseString(xml_string)
    
    # Pretty-print the DOM object into a formatted XML string
    pretty_xml_as_string = dom.toprettyxml(indent="  ")

    # Save the pretty-printed XML string to the file
    with open(file_name, 'w', encoding='utf-8') as file:
        file.write(pretty_xml_as_string)


In [84]:
# Maze 1
maze_layout = [
    [[1, 0, 0, 1], [1, 0, 1, 0], [1, 0, 1, 0], [1, 0, 1, 0], [1, 1, 0, 0]],  # First row with North walls
    [[0, 1, 0, 1], [1, 0, 1, 1], [1, 0, 1, 0], [1, 0, 1, 0], [0, 1, 1, 0]],  # Second row with South and West walls
    [[0, 0, 1, 1], [1, 0, 1, 0], [1, 0, 1, 0], [1, 0, 1, 0], [1, 1, 0, 0]],  # Third row
    [[1, 0, 0, 1], [1, 0, 1, 0], [1, 0, 1, 0], [1, 0, 1, 0], [1, 1, 0, 0]],  # Fourth row
    [[0, 0, 1, 1], [1, 0, 1, 0], [1, 0, 1, 0], [1, 0, 1, 0], [1, 1, 1, 0]]   # Fifth row
]

# Generate the walls
added_walls = generate_maze(maze_layout)

# Convert to LineStrings
line_segments = [LineString([wall[0], wall[1]]) for wall in added_walls]

# Combine parallel and touching walls
combined_walls = combine_walls_if_parallel_and_touching(line_segments)

# Example start and goal positions
start_positions = [{"x": 2.0, "y": -2.0, "theta": 3.14}]
goals = [{"id": 1, "x": -1.0, "y": 1.0}]

# Convert to XML
xml_output = walls_and_positions_to_xml(combined_walls, start_positions=start_positions, goals=goals)

# Output the XML
print(xml_output)

save_maze_to_file(xml_output,'../../WebotsSim/worlds/Fall24/maze1')

<world><wall x1="-2.5" y1="2.5" x2="2.5" y2="2.5" /><wall x1="-2.5" y1="2.5" x2="-2.5" y2="-2.5" /><wall x1="-1.5" y1="1.5" x2="1.5" y2="1.5" /><wall x1="2.5" y1="2.5" x2="2.5" y2="-2.5" /><wall x1="-1.5" y1="1.5" x2="-1.5" y2="0.5" /><wall x1="-1.5" y1="0.5" x2="2.5" y2="0.5" /><wall x1="-2.5" y1="-0.5" x2="2.5" y2="-0.5" /><wall x1="-1.5" y1="-1.5" x2="2.5" y2="-1.5" /><wall x1="-2.5" y1="-2.5" x2="2.5" y2="-2.5" /><startPositions><pos x="2.0" y="-2.0" theta="3.14" /></startPositions><goal id="1" x="-1.0" y="1.0" /></world>


In [85]:
# Maze 2
maze_layout = [
    [[1, 0, 1, 1], [1, 0, 1, 0], [1, 0, 1, 0], [1, 0, 1, 0], [1, 0, 1, 0], [1, 0, 1, 0], [1, 1, 1, 0]],
]


# Generate the walls
added_walls = generate_maze(maze_layout)

# Convert to LineStrings
line_segments = [LineString([wall[0], wall[1]]) for wall in added_walls]

# Combine parallel and touching walls
combined_walls = combine_walls_if_parallel_and_touching(line_segments)

# Example start and goal positions
start_positions = [{"x": -3.0, "y": 0.0, "theta": 0}]
goals = [{"id": 1, "x": 3.0, "y": 0.0}]

# Convert to XML
xml_output = walls_and_positions_to_xml(combined_walls, start_positions=start_positions, goals=goals)

# Output the XML
print(xml_output)

save_maze_to_file(xml_output,'../../WebotsSim/worlds/Fall24/maze2')

<world><wall x1="-3.5" y1="0.5" x2="3.5" y2="0.5" /><wall x1="-3.5" y1="-0.5" x2="3.5" y2="-0.5" /><wall x1="-3.5" y1="0.5" x2="-3.5" y2="-0.5" /><wall x1="3.5" y1="0.5" x2="3.5" y2="-0.5" /><startPositions><pos x="-3.0" y="0.0" theta="0" /></startPositions><goal id="1" x="3.0" y="0.0" /></world>


In [86]:
# Maze 3
maze_layout = [
    [[1,0,0,1],[1,0,1,0],[1,0,1,0],[1,0,1,0],[1,1,0,0]],
    [[0,1,0,1],[1,0,1,1],[1,0,1,0],[1,0,1,0],[0,1,1,0]],
    [[0,0,1,1],[1,0,1,0],[1,0,1,0],[1,0,1,0],[1,1,0,0]],
    [[1,0,0,1],[1,0,1,0],[1,0,1,0],[1,0,1,0],[0,1,1,0]],
    [[0,0,1,1],[1,0,1,0],[1,0,1,0],[1,0,1,0],[1,1,1,0]],
]


# Generate the walls
added_walls = generate_maze(maze_layout)

# Convert to LineStrings
line_segments = [LineString([wall[0], wall[1]]) for wall in added_walls]

# Combine parallel and touching walls
combined_walls = combine_walls_if_parallel_and_touching(line_segments)

# Example start and goal positions
start_positions = [{"x": 2, "y": -2, "theta": 3.14}]
goals = [{"id": 1, "x": -1, "y": 1}]

# Convert to XML
xml_output = walls_and_positions_to_xml(combined_walls, start_positions=start_positions, goals=goals)

# Output the XML
print(xml_output)

save_maze_to_file(xml_output,'../../WebotsSim/worlds/Fall24/maze3')

<world><wall x1="-2.5" y1="2.5" x2="2.5" y2="2.5" /><wall x1="-2.5" y1="2.5" x2="-2.5" y2="-2.5" /><wall x1="-1.5" y1="1.5" x2="1.5" y2="1.5" /><wall x1="2.5" y1="2.5" x2="2.5" y2="-2.5" /><wall x1="-1.5" y1="1.5" x2="-1.5" y2="0.5" /><wall x1="-1.5" y1="0.5" x2="2.5" y2="0.5" /><wall x1="-2.5" y1="-0.5" x2="1.5" y2="-0.5" /><wall x1="-1.5" y1="-1.5" x2="2.5" y2="-1.5" /><wall x1="-2.5" y1="-2.5" x2="2.5" y2="-2.5" /><startPositions><pos x="2" y="-2" theta="3.14" /></startPositions><goal id="1" x="-1" y="1" /></world>


In [87]:
# Maze 4
maze_layout = [
    [[1,0,0,1],[1,0,1,0],[1,0,1,0],[1,0,1,0],[1,1,0,0]],
    [[0,1,0,1],[1,0,1,1],[1,0,1,0],[1,0,1,0],[0,1,1,0]],
    [[0,0,0,1],[1,0,1,0],[1,0,1,0],[1,0,1,0],[1,1,0,0]],
    [[0,0,0,1],[1,0,1,0],[1,0,1,0],[1,0,1,0],[0,1,0,0]],
    [[0,0,1,1],[1,0,1,0],[1,0,1,0],[1,0,1,0],[0,1,1,0]],
]


# Generate the walls
added_walls = generate_maze(maze_layout)

# Convert to LineStrings
line_segments = [LineString([wall[0], wall[1]]) for wall in added_walls]

# Combine parallel and touching walls
combined_walls = combine_walls_if_parallel_and_touching(line_segments)

# Example start and goal positions
start_positions = [{"x": 2, "y": -2, "theta": 3.14}]
goals = [{"id": 1, "x": -1, "y": 1}]

# Convert to XML
xml_output = walls_and_positions_to_xml(combined_walls, start_positions=start_positions, goals=goals)

# Output the XML
print(xml_output)

save_maze_to_file(xml_output,'../../WebotsSim/worlds/Fall24/maze4')

<world><wall x1="-2.5" y1="2.5" x2="2.5" y2="2.5" /><wall x1="-2.5" y1="2.5" x2="-2.5" y2="-2.5" /><wall x1="-1.5" y1="1.5" x2="1.5" y2="1.5" /><wall x1="2.5" y1="2.5" x2="2.5" y2="-2.5" /><wall x1="-1.5" y1="1.5" x2="-1.5" y2="0.5" /><wall x1="-1.5" y1="0.5" x2="2.5" y2="0.5" /><wall x1="-1.5" y1="-0.5" x2="1.5" y2="-0.5" /><wall x1="-1.5" y1="-1.5" x2="1.5" y2="-1.5" /><wall x1="-2.5" y1="-2.5" x2="2.5" y2="-2.5" /><startPositions><pos x="2" y="-2" theta="3.14" /></startPositions><goal id="1" x="-1" y="1" /></world>


In [90]:
# Maze 9
# 17x17 Maze Layout
maze_layout = np.ones((17, 17, 4), dtype=int)  # Initialize all walls present (1s)

# Create a simple path system
for i in range(1, 16):
    if i % 2 == 1:  # For odd rows/columns, create paths
        maze_layout[i, 1:-1, 2] = 0  # Clear south walls for horizontal paths
        maze_layout[i + 1, 1:-1, 0] = 0  # Clear north walls for horizontal paths
        maze_layout[1:-1, i, 1] = 0  # Clear east walls for vertical paths
        maze_layout[1:-1, i + 1, 3] = 0  # Clear west walls for vertical paths

# Introduce more openings to ensure every cell is accessible
for i in range(1, 16, 2):
    for j in range(1, 16, 2):
        if np.random.rand() > 0.5:  # Randomly remove some walls
            maze_layout[i, j, 2] = 0  # Remove south wall
            maze_layout[i + 1, j, 0] = 0  # Remove north wall of the cell below
        if np.random.rand() > 0.5:
            maze_layout[i, j, 1] = 0  # Remove east wall
            maze_layout[i, j + 1, 3] = 0  # Remove west wall of the cell to the right

# Ensure access to the center
maze_layout[8, 8, 1] = 0  # Remove east wall at the center
maze_layout[8, 9, 3] = 0  # Remove west wall to the right of the center




# Generate the walls
added_walls = generate_maze(maze_layout)

# Convert to LineStrings
line_segments = [LineString([wall[0], wall[1]]) for wall in added_walls]

# Combine parallel and touching walls
combined_walls = combine_walls_if_parallel_and_touching(line_segments)

# Example start and goal positions
start_positions = [{"x": -1.5, "y": 1.5, "theta": 3.14}]
goals = [{"id": 1, "x": 2.5, "y": -2.5}]

# Convert to XML
xml_output = walls_and_positions_to_xml(combined_walls, start_positions=start_positions, goals=goals)


save_maze_to_file(xml_output,'../../WebotsSim/worlds/Fall24/maze9')

<world><wall x1="-8.5" y1="8.5" x2="8.5" y2="8.5" /><wall x1="-7.5" y1="8.5" x2="-7.5" y2="-8.5" /><wall x1="-8.5" y1="7.5" x2="8.5" y2="7.5" /><wall x1="-8.5" y1="8.5" x2="-8.5" y2="-8.5" /><wall x1="-6.5" y1="8.5" x2="-6.5" y2="7.5" /><wall x1="-5.5" y1="8.5" x2="-5.5" y2="-8.5" /><wall x1="-4.5" y1="8.5" x2="-4.5" y2="7.5" /><wall x1="-3.5" y1="8.5" x2="-3.5" y2="-8.5" /><wall x1="-2.5" y1="8.5" x2="-2.5" y2="7.5" /><wall x1="-1.5" y1="8.5" x2="-1.5" y2="-8.5" /><wall x1="-0.5" y1="8.5" x2="-0.5" y2="7.5" /><wall x1="0.5" y1="8.5" x2="0.5" y2="0.5" /><wall x1="1.5" y1="8.5" x2="1.5" y2="7.5" /><wall x1="2.5" y1="8.5" x2="2.5" y2="-8.5" /><wall x1="3.5" y1="8.5" x2="3.5" y2="7.5" /><wall x1="4.5" y1="8.5" x2="4.5" y2="-8.5" /><wall x1="5.5" y1="8.5" x2="5.5" y2="7.5" /><wall x1="6.5" y1="8.5" x2="6.5" y2="-8.5" /><wall x1="7.5" y1="8.5" x2="7.5" y2="7.5" /><wall x1="8.5" y1="8.5" x2="8.5" y2="-8.5" /><wall x1="-8.5" y1="6.5" x2="-7.5" y2="6.5" /><wall x1="7.5" y1="6.5" x2="8.5" y2="6